In [2]:
from instruments_recognition.importing_data import *
from instruments_recognition.plotting import *

import random

In [174]:
### Import transformed data (features) from file

instrument1 = 'flute_philharmonia'
instrument1te = 'flute_bach'
instrument2 = 'oboe_philharmonia'
instrument2te = 'oboe_bach'


# input: list of lists
# output: only lists that do not contain any nans
def cleannan2(l):
    return [x for x in l if np.isfinite(x).all()]

def cleannan_and_warn(l):
    res = []
    cleared = 0
    for x in l :
        if np.isfinite(x).all() :
            res.append(x)
        else :
            cleared += 1
    print('killed ' + str(cleared) + ' nans')
    return res

def just_release(l) :
    return l[1]

def flatten(l) :
    res = []
    list(map(res.extend, l))
    return res

def take_n_to_m_windows(l,n,m):
    return  l[n:m]
    
def take_n_harmonics(l,n):
    """
    input: a list of vector of volumes, a vector of volumes for each window
    """
    return [ w[:n] for w in l ]

def import_and_clean_energies(datafile, howmanyharmonics = 9) :
    energiesf = open(datafile, 'rb')
    energies = np.load(energiesf)
    #print(energies)
    #print(str(type(howmanyharmonics)))
    energiesflattened = [ flatten(take_n_harmonics(take_n_to_m_windows(energy,0,3), howmanyharmonics)) for energy in energies ]
    volumes = cleannan_and_warn(energiesflattened)
    #shuffle the input so we train on a random set of the input (the shuffle is done in place )
    random.seed(1)
    random.shuffle(volumes)
    return volumes

volumes1 = import_and_clean_energies('transformed_data/' + instrument1 + '_multiwindow_volumes')
volumes1te = import_and_clean_energies('transformed_data/' + instrument1te + '_multiwindow_volumes')
volumes2 = import_and_clean_energies('transformed_data/' + instrument2 + '_multiwindow_volumes')
volumes2te = import_and_clean_energies('transformed_data/' + instrument2te + '_multiwindow_volumes')

 

killed 0 nans
killed 0 nans
killed 0 nans
killed 0 nans


In [180]:
### LEARN 2

from sklearn import neighbors, svm

#fractiontrain = 9./10.
fractiontrain = 5./6.
#fractiontrain = 4./5.
#fractiontrain = 1./100.

#training examples and testing examples
volumes1.extend(volumes1te)
volumes2.extend(volumes2te)
trainvolumes1 = volumes1[:int(len(volumes1)*fractiontrain)]
trainvolumes2 = volumes2[:int(len(volumes2)*fractiontrain)]
testvolumes1 = volumes1[int(len(volumes1)*fractiontrain):len(volumes1)]
testvolumes2 = volumes2[int(len(volumes2)*fractiontrain):len(volumes2)]

#trainvolumes1 = volumes1
#trainvolumes2 = volumes2
#testvolumes1 = volumes1te
#testvolumes2 = volumes2te

howmanytrain1 = len(trainvolumes1)
print("# training instrument 1: " + str(howmanytrain1))
howmanytrain2 = len(trainvolumes2)
print("# training instrument 2: " + str(howmanytrain2))
howmanytrain = howmanytrain1 + howmanytrain2

howmanytest1 = len(testvolumes1)
howmanytest2 = len(testvolumes2)
print("# testing instrument 1: " + str(howmanytest1))
print("# testing instrument 2: " + str(howmanytest2))


training_ex = trainvolumes1 + trainvolumes2
labels = [ 1 for i in range(0,howmanytrain1) ] + [ 0 for i in range(0,howmanytrain2) ]


#########################

## assert: there are no testing samples in the training set

def is_in(e,l):
    res = False
    for v in l:
        if all([f1 == f2 for f1, f2 in zip(e,v)]) :
            res = True
    return res
            
#testing: is_in([1.],[[0.],[0.,4,3],[1.]])
    
alltests = testvolumes1 + testvolumes2

datasnooping = any([ is_in(test, training_ex) for test in alltests ])
if datasnooping :
    print("WARNING: you are testing on the training set")
print("doing data snooping? " + str(datasnooping))

# training instrument 1: 730
# training instrument 2: 1070
# testing instrument 1: 146
# testing instrument 2: 214
doing data snooping? False


In [183]:
### Learning algorithm

# SVN does not seem to work
clf = svm.SVC()
clf.fit(training_ex, labels) 


## nearest neighboor
#n_neighbors = 200
#clf = neighbors.KNeighborsClassifier(n_neighbors)
#clf.fit(training_ex, labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [184]:
### COMPUTING ERROR

print("training with: " + str(howmanytrain))
print("testing with: " + str(howmanytest1 + howmanytest2))

#goods1 = list(clf.predict(testvolumes1))
goods2 = [ 1-x for x in clf.predict(testvolumes2) ]

error = 1. - np.average(goods1+goods2)

print("error: " + str(error))


training with: 1800
testing with: 360
error: 0.0408163265306


In [10]:
### OLD Import transformed data (features) from file

instrument1 = "flute"
instrument2 = "oboe"


energylbl1af = open('transformed_data/' + instrument1 + '_energylbla_experiment', 'rb')
energylbl1rf = open('transformed_data/' + instrument1 + '_energylblr_experiment', 'rb')
energylbl2af = open('transformed_data/' + instrument2 + '_energylbla_experiment', 'rb')
energylbl2rf = open('transformed_data/' + instrument2 + '_energylblr_experiment', 'rb')

energylbl1a_withmore = np.load(energylbl1af)
energylbl1r_withmore = np.load(energylbl1rf)
energylbl2a_withmore = np.load(energylbl2af)
energylbl2r_withmore = np.load(energylbl2rf)

#print(energylbl1a_withmore[0])


In [40]:

### LEARN 1


howmanyharmonics = 6

# forget the frequencies
# and forget the first volume which is always 1
def only_volumes(energylbl) :
    return list(map(lambda l : l[1][1:howmanyharmonics+1], energylbl))


# input: list of lists
# output: only lists that do not contain any nans
def cleannan(l):
    return [x for x in l if np.isfinite(x).all()]


energylbl1a = only_volumes(energylbl1a_withmore)
energylbl1r = only_volumes(energylbl1r_withmore)
energylbl2a = only_volumes(energylbl2a_withmore)
energylbl2r = only_volumes(energylbl2r_withmore)

# how many harmonics do we have
print(str(len(energylbl1a_withmore[1][1][1:])) + " harmonics avaiable")

# we use attack and release and the difference of volume between the two
#energylbl1 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]
#energylbl1 = [ (l1 + l2) for l1, l2 in zip(energylbl1a,energylbl1r)]
#energylbl2 = [ (l1 + l2) for l1, l2 in zip(energylbl2a,energylbl2r)]

# but in the attack we sometimes fail to recognize the tonic (it is mainly confused with the octave)
# so we can use the release and the difference of volume between attack and release
#energylbl1 = [ (l1 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]

# *but* it seems to work better with just l2
energylbl1 = energylbl1r
energylbl2 = energylbl2r


volumes1 = cleannan2(energylbl1)
volumes2 = cleannan2(energylbl2)
#shuffle the input so we train on a random set of the input (the shuffle is done in place )
random.seed(1)
random.shuffle(volumes1)
random.shuffle(volumes2)




#nthvolume = [ map(lambda l : l[n], only_volumes) for n in range(0,num_harmonics) ]
#
#nthvolumecleaned = map(cleannan, nthvolume)
#
#averages = map(average, nthvolumecleaned)
#
#minusaverages = [ map(lambda m : m-averages[n], nthvolumecleaned[n]) for n in range(0,num_harmonics) ]
#
#variances = map(rms, minusaverages)
#
#variances, averages

#print(vd1[2])

8 harmonics avaiable


In [ ]:

## forget the frequencies
## and forget the first volume which is always 1
#def only_volumes2(energies) :
#    return [ [ l[1:howmanyharmonics+1] for l in energiess ] for energiess in energies ]



#####################energies1f = open('transformed_data/' + instrument1 + '_multiwindow_volumes', 'rb')
#####################energies1fte = open('transformed_data/' + instrument1te + '_multiwindow_volumes', 'rb')
#####################energies2f = open('transformed_data/' + instrument2 + '_multiwindow_volumes', 'rb')
#####################
#####################energies1_withmore = np.load(energies1f)
#####################energies1te_withmore = np.load(energies1fte)
#####################energies2_withmore = np.load(energies2f)


### LEARN 1


#print("hola " + str(flatten([[1,2,3],[4,5,6]])))
#print([1,2,3].extend([4,5,6]))


#####################energies1 = only_volumes2(energies1_withmore)
#####################energies1te = only_volumes2(energies1te_withmore)
#####################energies2 = only_volumes2(energies2_withmore)

## how many harmonics do we have
#print(str(len(energylbl1a_withmore[1][1][1:])) + " harmonics avaiable")

# we use attack and release and the difference of volume between the two
#energylbl1 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]
#energylbl1 = [ (l1 + l2) for l1, l2 in zip(energylbl1a,energylbl1r)]
#energylbl2 = [ (l1 + l2) for l1, l2 in zip(energylbl2a,energylbl2r)]

# but in the attack we sometimes fail to recognize the tonic (it is mainly confused with the octave)
# so we can use the release and the difference of volume between attack and release
#energylbl1 = [ (l1 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]

# *but* it seems to work better with just l2
#####################energies1flattened = list(map(flatten,energies1))
#####################energies1teflattened = list(map(flatten,energies1te))
#####################energies2flattened = list(map(flatten,energies2))
#####################
#####################
#####################volumes1 = cleannan2(energies1flattened)
#####################volumes1te = cleannan2(energies1teflattened)
#####################volumes2 = cleannan2(energies2flattened)
######################shuffle the input so we train on a random set of the input (the shuffle is done in place )
#####################random.seed(1)
#####################random.shuffle(volumes1)
#####################random.shuffle(volumes1te)
#####################random.shuffle(volumes2)




#nthvolume = [ map(lambda l : l[n], only_volumes) for n in range(0,num_harmonics) ]
#
#nthvolumecleaned = map(cleannan, nthvolume)
#
#averages = map(average, nthvolumecleaned)
#
#minusaverages = [ map(lambda m : m-averages[n], nthvolumecleaned[n]) for n in range(0,num_harmonics) ]
#
#variances = map(rms, minusaverages)
#
#variances, averages

#print(vd1[2])